In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def test_cuts(x, col, thr, n=4):
    display(pd.cut(x[x[col] < thr][col], bins=n).value_counts(sort=False))
    display(pd.qcut(x[x[col] < thr][col], q=n).value_counts(sort=False))

In [3]:
orig_df = pd.read_csv('../../state_CA.csv').drop_duplicates()

/tmp/ipykernel_90307/3013834244.py:1: DtypeWarning: Columns (22,23,24,26,27,28,29,30,31,32,33,38,43,44,46,77,78,79,80) have mixed types. Specify dtype option on import or set low_memory=False.
  orig_df = pd.read_csv('../../state_CA.csv').drop_duplicates()


In [4]:
df = orig_df.drop(
    [
        "activity_year",
        "lei",
        "derived_msa-md",
        "state_code",
        "census_tract",
        "aus-2",
        "aus-3",
        "aus-4",
        "aus-5",
        "denial_reason-1",
        "denial_reason-2",
        "denial_reason-2",
        "denial_reason-3",
        "denial_reason-4",
        "reverse_mortgage",
        "hoepa_status",
        "negative_amortization",
        "derived_dwelling_category",
        "other_nonamortizing_features",
        "manufactured_home_secured_property_type",
        "manufactured_home_land_property_interest",
        "initially_payable_to_institution",
    ],
    axis = 1
)
df = df.drop([x for x in df.columns if x.endswith('observed')], axis=1)
df = df.drop([x for x in df.columns if 'tract' in x or 'ffiec' in x], axis=1)
df = df[df != 'Exempt']

In [5]:
# 1 - Loan originated
# 2 - Application approved but not accepted
# 3 - Application denied
# 4 - Application withdrawn by applicant
# 5 - File closed for incompleteness
# 6 - Purchased loan
# 7 - Preapproval request denied
# 8 - Preapproval request approved but not accepted
display(df['action_taken'].value_counts())
df = df[df['action_taken'].isin([1, 2, 3])]
df["action_taken"] = df["action_taken"].map({1: 'Yes', 2: 'Yes', 3: 'No'})
display(df['action_taken'].value_counts())

action_taken
1    798306
3    242203
4    233023
6    145043
5     85203
2     40347
8      4469
7      1409
Name: count, dtype: int64

action_taken
Yes    838653
No     242203
Name: count, dtype: int64

In [6]:
# 1 - Male
# 2 - Female
# 3 - Information not provided by applicant in mail, internet, or telephone application
# 4 - Not applicable
# 6 - Applicant selected both male and female
display(df['applicant_sex'].value_counts())
df = df[df['applicant_sex'].isin([1, 2])]
df['applicant_sex'] = df['applicant_sex'].map({1: 'male', 2: 'female'})
display(df['applicant_sex'].value_counts())

applicant_sex
1    627136
2    334119
3     99086
4     19674
6       841
Name: count, dtype: int64

applicant_sex
male      627136
female    334119
Name: count, dtype: int64

In [7]:
# 1 - Male
# 2 - Female
# 3 - Information not provided by applicant in mail, internet, or telephone application
# 4 - Not applicable
# 5 - no co-applicant
# 6 - Applicant selected both male and female
display(df['co-applicant_sex'].value_counts())
df = df[df['co-applicant_sex'].isin([1, 2, 5])]
df['co-applicant_sex'] = df['co-applicant_sex'].map({1: 'male', 2: 'female', 5: 'none'})
display(df['co-applicant_sex'].value_counts())

co-applicant_sex
5    515399
2    300301
1    133036
3      9912
4      2259
6       348
Name: count, dtype: int64

co-applicant_sex
none      515399
female    300301
male      133036
Name: count, dtype: int64

In [8]:
# Loan type - keeping 1/2/3
# 1 - Conventional (not insured or guaranteed by FHA, VA, RHS, or FSA)
# 2 - Federal Housing Administration insured (FHA)
# 3 - Veterans Affairs guaranteed (VA)
# 4 - USDA Rural Housing Service or Farm Service Agency guaranteed (RHS or FSA)
display(df['loan_type'].value_counts())
df["loan_type"] = df["loan_type"].map(
    {1: "Conventional", 2: "FHA", 3: "VA", 4: "RHS/FSA"}
)
df = df[df["loan_type"] != "RHS/FSA"]
display(df['loan_type'].value_counts())

loan_type
1    824901
2     81749
3     41228
4       858
Name: count, dtype: int64

loan_type
Conventional    824901
FHA              81749
VA               41228
Name: count, dtype: int64

In [9]:
display(df['purchaser_type'].value_counts())
df["purchaser_type"] = df["purchaser_type"].map(
    {
        0: "NotApplicable",
        1: "GSE",
        2: "GSE",
        3: "GSE",
        4: "Farmer Mac",
        5: "Private",
        6: "Bank",
        71: "CU/MC/FC",
        72: "LifeInsurance",
        8: "Affiliate",
        9: "Other",
    }
)
df = df[df["purchaser_type"] != "Other"]
display(df['purchaser_type'].value_counts())

purchaser_type
0     517428
1     133218
3      92471
71     70968
2      59515
6      31272
9      15468
5      15035
8       6826
72      5677
Name: count, dtype: int64

purchaser_type
NotApplicable    517428
GSE              285204
CU/MC/FC          70968
Bank              31272
Private           15035
Affiliate          6826
LifeInsurance      5677
Name: count, dtype: int64

In [10]:
display(df["derived_loan_product_type"].value_counts())
df = df[
    (df["derived_loan_product_type"] != "FHA:Subordinate Lien")
    & (df["derived_loan_product_type"] != "VA:Subordinate Lien")
]
display(df["derived_loan_product_type"].value_counts())

derived_loan_product_type
Conventional:First Lien          585749
Conventional:Subordinate Lien    226729
FHA:First Lien                    78976
VA:First Lien                     40927
FHA:Subordinate Lien                 28
VA:Subordinate Lien                   1
Name: count, dtype: int64

derived_loan_product_type
Conventional:First Lien          585749
Conventional:Subordinate Lien    226729
FHA:First Lien                    78976
VA:First Lien                     40927
Name: count, dtype: int64

In [11]:
display(df["loan_purpose"].value_counts())
df["loan_purpose"] = df["loan_purpose"].map(
    {
        1: "Home purchase",
        2: "Home improvement",
        31: "Refinancing",
        32: "Cash-out refinancing",
        4: "Other",
        5: "Not applicable",
    }
)
df = df[df["loan_purpose"] != "Not applicable"]
display(df["loan_purpose"].value_counts(dropna=False))

loan_purpose
1     354780
32    229046
31    134248
2     121023
4      93259
5         25
Name: count, dtype: int64

loan_purpose
Home purchase           354780
Cash-out refinancing    229046
Refinancing             134248
Home improvement        121023
Other                    93259
Name: count, dtype: int64

In [12]:
df["preapproval"] = df["preapproval"].map({1: 'Yes', 2: 'No'})
display(df["preapproval"].value_counts(dropna=False))

preapproval
No     911931
Yes     20425
Name: count, dtype: int64

In [13]:
df["lien_status"] = df["lien_status"].map(
    {
        1: "First",
        2: "Subordinate",
    }
)
display(df["lien_status"].value_counts(dropna=False))

lien_status
First          705629
Subordinate    226727
Name: count, dtype: int64

In [14]:
display(df["business_or_commercial_purpose"].value_counts())
df["business_or_commercial_purpose"] = (
    df["business_or_commercial_purpose"].map({1: 'Yes', 2: 'No'})
)
display(df["business_or_commercial_purpose"].value_counts(dropna=False))

business_or_commercial_purpose
2       881898
1        44911
1111      5547
Name: count, dtype: int64

business_or_commercial_purpose
No     881898
Yes     44911
NaN      5547
Name: count, dtype: int64

In [15]:
display(df["open-end_line_of_credit"].value_counts())
df["open-end_line_of_credit"] = (
    df["open-end_line_of_credit"].map({1: 'Yes', 2: 'No'})
)
display(df["open-end_line_of_credit"].value_counts())

open-end_line_of_credit
2       714526
1       212525
1111      5305
Name: count, dtype: int64

open-end_line_of_credit
No     714526
Yes    212525
Name: count, dtype: int64

In [16]:
test_cuts(df, 'loan_amount', 2e6, n=4)
df = df[df['loan_amount'] <= 2e6]

loan_amount
(3010.0, 502500.0]        603903
(502500.0, 1000000.0]     249845
(1000000.0, 1497500.0]     47108
(1497500.0, 1995000.0]     16068
Name: count, dtype: int64

loan_amount
(4999.999, 195000.0]     231417
(195000.0, 365000.0]     227663
(365000.0, 585000.0]     232722
(585000.0, 1995000.0]    225122
Name: count, dtype: int64

In [17]:
for col in df.columns:
    is_null = df[col].isnull().sum()
    if is_null > 0:
        print(col, is_null)

county_code 1656
conforming_loan_limit 824
open-end_line_of_credit 5227
business_or_commercial_purpose 5469
loan_to_value_ratio 59959
interest_rate 214029
rate_spread 280535
total_loan_costs 425780
total_points_and_fees 903926
origination_charges 414476
discount_points 650226
lender_credits 724024
loan_term 27838
prepayment_penalty_term 830426
intro_rate_period 651549
property_value 25281
multifamily_affordable_units 916217
income 26263
debt_to_income_ratio 58905
applicant_ethnicity-1 568
applicant_ethnicity-2 802011
applicant_ethnicity-3 915074
applicant_ethnicity-4 916887
applicant_ethnicity-5 916921
co-applicant_ethnicity-1 209
co-applicant_ethnicity-2 865322
co-applicant_ethnicity-3 916188
co-applicant_ethnicity-4 916908
co-applicant_ethnicity-5 916923
applicant_race-1 286
applicant_race-2 792489
applicant_race-3 907996
applicant_race-4 916073
applicant_race-5 916705
co-applicant_race-1 99
co-applicant_race-2 861008
co-applicant_race-3 912895
co-applicant_race-4 916573
co-applicant

In [18]:
drop_df = df.copy()

above_threshold_null_cols = [x for x in drop_df.columns if drop_df[x].isnull().sum() > 80000]
drop_df = drop_df.drop(above_threshold_null_cols, axis=1)

print(drop_df.shape)
drop_df = drop_df.dropna()
print(drop_df.shape)

for col in drop_df.columns:
    is_null = drop_df[col].isnull().sum()
    if is_null > 0:
        print(col, is_null)

drop_df = drop_df.drop(['applicant_credit_score_type', 'co-applicant_credit_score_type'], axis=1)
print(drop_df.shape)

(916924, 38)
(822957, 38)
(822957, 36)


In [19]:
mapped_df = drop_df.copy()

# 1 - Desktop Underwriter (DU)
# 2 - Loan Prospector (LP) or Loan Product Advisor
# 3 - Technology Open to Approved Lenders (TOTAL) Scorecard
# 4 - Guaranteed Underwriting System (GUS)
# 5 - Other
# 6 - Not applicable
# 7 - Internal Proprietary System
# 1111 - Exempt
mapped_df["aus-1"] = (
    mapped_df["aus-1"].map({1: "DU", 2: "LP", 3: "TOTAL", 6: "NotApplicable", 7: "IPS"})
)
mapped_df = mapped_df.dropna(subset=["aus-1"], axis=0)

mapped_df["total_units"] = pd.to_numeric(mapped_df["total_units"])
mapped_df["interest_only_payment"] = mapped_df["interest_only_payment"].map(
    {1: 'Yes', 2: 'No'}
)
mapped_df["construction_method"] = mapped_df["construction_method"].map(
    {1: "site", 2: "manufactured"}
)
mapped_df["submission_of_application"] = mapped_df["submission_of_application"].map(
    {1: "direct", 2: "indirect"}
)

mapped_df["balloon_payment"] = mapped_df["balloon_payment"].map({1: 'Yes', 2: 'No'})

mapped_df["occupancy_type"] = mapped_df["occupancy_type"].map(
    {1: "mainres", 2: "secondres", 3: "investproperty"}
)

mapped_df["applicant_ethnicity-1"] = mapped_df["applicant_ethnicity-1"].map({
    1: 'Hispanic/Latino',
    2: 'Not Hispanic/Latino',
    3: 'Not Provided',
    4: 'NotApplicable',
    11: 'Hispanic/Latino', # Mexican
    12: 'Hispanic/Latino', # Puerto Rico
    13: 'Hispanic/Latino', # Cuban
    14: 'Hispanic/Latino', # other Hispanic or Latino
})
mapped_df = mapped_df[mapped_df["applicant_ethnicity-1"] != 'NotApplicable']

mapped_df["co-applicant_ethnicity-1"] = mapped_df["co-applicant_ethnicity-1"].map({
    1: 'Hispanic/Latino',
    2: 'Not Hispanic/Latino',
    3: 'Not Provided',
    4: 'NotApplicable',
    5: 'none',
    11: 'Hispanic/Latino', # Mexican
    12: 'Hispanic/Latino', # Puerto Rico
    13: 'Hispanic/Latino', # Cuban
    14: 'Hispanic/Latino', # other Hispanic or Latino
})
mapped_df = mapped_df[mapped_df["co-applicant_ethnicity-1"] != 'NotApplicable']

mapped_df["applicant_race-1"] = mapped_df["applicant_race-1"].apply(lambda x: int(str(x)[:1])).map(
    {
        1: 'American Indian',
        2: 'Asian',
        3: 'Black',
        4: 'Pacific Islander',
        5: 'White',
        6: 'Not Provided',
        7: 'NotApplicable',
    }
)
mapped_df = mapped_df[mapped_df["applicant_race-1"] != 'NotApplicable']

mapped_df["co-applicant_race-1"] = mapped_df["co-applicant_race-1"].apply(lambda x: int(str(x)[:1])).map(
    {
        1: 'American Indian',
        2: 'Asian',
        3: 'Black',
        4: 'Pacific Islander',
        5: 'White',
        6: 'Not Provided',
        7: 'NotApplicable',
        8: 'none',
    }
)
mapped_df = mapped_df[mapped_df["co-applicant_race-1"] != 'NotApplicable']

mapped_df["loan_to_value_ratio"] = pd.to_numeric(mapped_df["loan_to_value_ratio"])
mapped_df["loan_term"] = pd.to_numeric(mapped_df["loan_term"]) / 12

mapped_df['property_value'] = pd.to_numeric(mapped_df['property_value'])
mapped_df = mapped_df[mapped_df["property_value"] <= 3e6]

mapped_df['income'] *= 1000
mapped_df = mapped_df[(mapped_df['income'] > 0) & (mapped_df['income'] < 1e6)]

mapped_df["debt_to_income_ratio"] = mapped_df["debt_to_income_ratio"].replace(
    {
        "<20%": "10",
        "20%-<30%": "25",
        "30%-<36%": "33",
        "50%-60%": "55",
        ">60%": "70"
     }
).astype(float)

mapped_df["co-applicant_age"] = mapped_df["co-applicant_age"].replace('9999', 'none')
mapped_df = mapped_df[mapped_df["co-applicant_age"] != "8888"]

mapped_df = mapped_df.drop([
    'aus-1',
    'preapproval',
    'applicant_age_above_62',
    'submission_of_application',
], axis=1)

mapped_df = mapped_df.drop([x for x in mapped_df.columns if x.startswith('co-')], axis=1)
mapped_df = mapped_df.drop([x for x in mapped_df.columns if x.startswith('derived_')], axis=1)

In [20]:
mapped_df.columns

Index(['county_code', 'conforming_loan_limit', 'action_taken',
       'purchaser_type', 'loan_type', 'loan_purpose', 'lien_status',
       'open-end_line_of_credit', 'business_or_commercial_purpose',
       'loan_amount', 'loan_to_value_ratio', 'loan_term',
       'interest_only_payment', 'balloon_payment', 'property_value',
       'construction_method', 'occupancy_type', 'total_units', 'income',
       'debt_to_income_ratio', 'applicant_ethnicity-1', 'applicant_race-1',
       'applicant_sex', 'applicant_age'],
      dtype='object')

In [21]:
mapped_df = mapped_df.rename(columns = {
    'action_taken': 'success',
    'applicant_ethnicity-1': 'applicant_ethnicity',
    'applicant_race-1': 'applicant_race',
})

In [22]:
mapped_df.reset_index(drop=True).to_csv('../../mapped_df.csv')